In [1]:
import os
import polars as pl
import altair as alt

In [2]:
pl.Config.set_tbl_rows(300)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [3]:
df = pl.scan_parquet(
    "data/projevy.parquet"
).filter(
    pl.col('datum').dt.year() >= 1993
).filter(
    pl.col("komora_komplet").str.contains('Poslanecká') | pl.col('komora_komplet').str.contains('PČR, PS')
).with_columns(
    pl.col('mluvci_id').str.split('/sqw/detail.sqw?id=').alias('id')
).with_columns(
    pl.when(pl.col('id').len() > 2).then(pl.col('id').list.slice(1)).alias('id')
).explode(
    "id"
).with_columns(
    pl.col("text").str.count_matches(" ").alias("pocet_slov") # tady budou zlobit dvojité mezery, pohlídat při čištění
).sort(
    by="datum"
).with_columns(
    pl.when(pl.col("soubor") == "1996ps_stenprot_006schuz_s006024.htm").then(pl.lit(2006)).otherwise(pl.col('obdobi')).alias('obdobi')
).collect()

In [4]:
df.filter(pl.col("text").str.contains("(?i)dozimet")).head(5)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32
"""Poslanec Jaroslav Palas""","""1998ps_stenprot_026schuz_s026115.htm""","""/sqw/p.sqw?P=543""","""Pane předsedající, vážená vládo, dámy a pánové, dovolte mi, abych po dohodě s předkladatelem a zpravodajem k tomuto bodu předložil několik pozměňovací…",3,"""ps""",1998,26,115,2000-06-29,"""Parlament České republiky, Poslanecká sněmovna 1998 – 2002""","""26. schůze""",true,true,true,null,880
"""Poslanec Aleš Rozehnal""","""2002ps_stenprot_033schuz_s033080.htm""","""/sqw/detail.sqw?id=412""","""(9.50 hodin) Využití této suroviny na financování krvavých konfliktů je zcela mimo realitu i vzhledem k množství této suroviny dovážené. Podle mého n…",1,"""ps""",2002,33,80,2004-06-17,"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""","""33. schůze""",true,true,false,"""412""",722
"""Poslanec Vladimír Zlínský""","""2021ps_stenprot_009schuz_s009234.htm""","""/sqw/detail.sqw?id=6795""","""Vážená paní předsedající, děkuji za slovo. Vážený pane premiére, mohl byste vzhledem k možným rizikům jaderného konfliktu na našem území nebo v našem …",7,"""ps""",2021,9,234,2022-03-03,"""PČR, PS 2021-...""","""9. schůze""",true,true,true,"""6795""",196
"""Poslanec Tomio Okamura""","""2021ps_stenprot_030schuz_s030004.htm""","""/sqw/detail.sqw?id=6105""","""Vážené dámy a pánové, i díky podpisům poslanců hnutí SPD byla na dnešek svolána mimořádná schůze Poslanecké sněmovny s cílem projednat organizovaný zl…",3,"""ps""",2021,30,4,2022-07-07,"""PČR, PS 2021-...""","""30. schůze""",true,true,true,"""6105""",880
"""Poslanec Radim Fiala""","""2021ps_stenprot_030schuz_s030011.htm""","""/sqw/detail.sqw?id=5459""","""Dosud nevysvětlené, nesmírně závažné kauzy hnutí STAN, z nichž mnohé mají protizákonný charakter, nás provázejí prakticky nonstop od posledních voleb.…",1,"""ps""",2021,30,11,2022-07-07,"""PČR, PS 2021-...""","""30. schůze""",true,true,false,"""5459""",1282


In [5]:
df.filter(pl.col("text").str.contains("(?i)vlastizrád")).head(5)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32
"""Poslanec Jaroslav Novák""","""1993ps_stenprot_002schuz_s002005.htm""",null,"""Vážený pane předsedo, nepřítomná vládo, dámy a pánové, díky publicitě, kterou mi v poslední době věnovaly sdělovací prostředky právě v souvislosti s p…",4,"""ps""",1993,2,5,1993-01-26,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""2. schůze""",true,true,true,null,567
"""Poslanec Jan Vik""","""1993ps_stenprot_033schuz_s033003.htm""","""/sqw/hlasy.sqw?G=2355""","""V prvé řadě se omlouvám panu poslanci Vorlíčkovi, ale myslím, že se dnes jedná především o mě, tak bych si také k tomu řekl rád včas svoje. Pane předs…",1,"""ps""",1993,33,3,1995-07-25,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""33. schůze""",true,true,true,null,1795
"""Poslanec Rudolf Šmucr""","""1996ps_stenprot_007schuz_s007012.htm""","""/sqw/hlasy.sqw?G=5978""","""Vážený pane předsedající, dámy a pánové, rád bych vás seznámil s obsahem mého vystoupení k návrhu zákona o státním rozpočtu České republiky na rok 199…",10,"""ps""",1996,7,12,1996-12-11,"""Parlament České republiky, Poslanecká sněmovna 1996 – 1998""","""7. schůze""",true,true,true,null,368
"""Poslanec Josef Krejsa""","""1996ps_stenprot_008schuz_s008006.htm""","""/sqw/hlasy.sqw?G=5978""","""Vážení i nevážení, poněvadž už pouhým projednáváním tohoto bodu už tím, že se vůbec dostal na pořad 8. schůze, že po vzoru gestapa či StB si dovolila …",11,"""ps""",1996,8,6,1997-02-05,"""Parlament České republiky, Poslanecká sněmovna 1996 – 1998""","""8. schůze""",true,true,true,null,1056
"""Poslanec Jaromír Kalus""","""1996ps_stenprot_008schuz_s008051.htm""","""/sqw/hlasy.sqw?G=5978""","""Vážený pane předsedající, dámy a pánové, v průběhu našeho diskusního maratónu zde byla přednesena řada globálních pohledů na celý komplex všech souvis…",31,"""ps""",1996,8,51,1997-02-12,"""Parlament České republiky, Poslanecká sněmovna 1996 – 1998""","""8. schůze""",true,true,true,null,1032


In [6]:
df.filter(pl.col("text").str.contains("(?i)endiaron")).head(5)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32
"""Poslankyně Věra Kovářová""","""2017ps_stenprot_040schuz_s040116.htm""","""/sqw/detail.sqw?id=6214""","""Vážený pane ministře, obracím se na vás se závažným tématem, a to je nedostupnost léků - dnes to již zaznělo - a tedy nedostupnost léků v lékárnách. P…",2,"""ps""",2017,40,116,2020-01-23,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""40. schůze""",true,true,true,"""6214""",234
"""Poslankyně Jana Mračková Vildumetzová""","""2021ps_stenprot_052schuz_s052108.htm""","""/sqw/detail.sqw?id=6471""","""Děkuji za slovo. Vážená paní místopředsedkyně, vážení členové vlády, kolegyně, kolegové, já bych si dovolila zařadit dva body. První bod s názvem Česk…",3,"""ps""",2021,52,108,2023-02-10,"""PČR, PS 2021-...""","""52. schůze""",true,true,true,"""6471""",1132
"""Poslanec Andrej Babiš""","""2021ps_stenprot_071schuz_s071311.htm""","""/sqw/detail.sqw?id=6150""","""Ano. Nezvládla to. Už máte ten seznam od paní Kovářové, abychom věděli ta slova? Ale váš kolega píše na Twitter K, víte, já jsem mluvil jenom P, víte,…",3,"""ps""",2021,71,311,2023-07-26,"""PČR, PS 2021-...""","""71. schůze""",true,true,true,"""6150""",942
"""Poslanec Andrej Babiš""","""2021ps_stenprot_074schuz_s074014.htm""","""/sqw/detail.sqw?id=6150""","""Takže to je ta politika. Kdo tu rozděluje tu společnost tedy? Kdo ji rozděluje? Ale ony to mají v genech, ty tradiční strany. Já si pamatuji za ČSSD, …",1,"""ps""",2021,74,14,2023-09-05,"""PČR, PS 2021-...""","""74. schůze""",true,true,false,"""6150""",1036
"""Poslanec Andrej Babiš""","""2021ps_stenprot_074schuz_s074016.htm""","""/sqw/detail.sqw?id=6150""","""Takže tohle jsme všechno dělali a samozřejmě jsme děkovali zdravotníkům, a dali jsme jim peníze, mimořádné odměny. Ve vládě jsme samozřejmě vypláceli …",1,"""ps""",2021,74,16,2023-09-05,"""PČR, PS 2021-...""","""74. schůze""",true,true,false,"""6150""",1057


In [7]:
df.filter(pl.col("text").str.contains("(?i)videohr")).head(5)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32
"""Ministryně obrany ČR Jana Černochová""","""2021ps_stenprot_079schuz_s079040.htm""","""https://www.vlada.cz/cz/clenove-vlady/jana-cernochova-191692/""","""V květnu 2023 mě také vláda pověřila k jednání o pořízení tanků Leopard A28, jejichž nákup plánuje Spolková republika Německo. Armádě České republiky …",1,"""ps""",2021,79,40,2023-10-17,"""PČR, PS 2021-...""","""79. schůze""",true,true,false,null,935
"""Poslanec Lubomír Brož""","""2021ps_stenprot_110schuz_s110005.htm""","""/sqw/detail.sqw?id=6725""","""Já bych jenom připomenul takové důležité věci a tam dal i svoje otazníky - budeme o tom určitě mluvit na výboru a ve druhém čtení. Rozšířila se ta obl…",1,"""ps""",2021,110,5,2024-07-10,"""PČR, PS 2021-...""","""110. schůze""",true,true,false,"""6725""",550
"""Poslanec Milan Brázdil""","""2021ps_stenprot_110schuz_s110005.htm""","""/sqw/detail.sqw?id=6185""","""Děkuji za slovo. Já už jsem se pana ministra ptal, ale přece jenom do éteru se zeptám. Pobídky pro tvůrce videoher... My žijeme v době, kdy děcka jsou…",3,"""ps""",2021,110,5,2024-07-10,"""PČR, PS 2021-...""","""110. schůze""",true,true,true,"""6185""",122
"""Ministr kultury ČR Martin Baxa""","""2021ps_stenprot_110schuz_s110006.htm""","""https://www.vlada.cz/cz/clenove-vlady/martin-baxa-191703/""","""Děkuji za slovo, pane místopředsedo. Děkuju za tu bohatou rozpravu v prvním čtení, za přednesené podněty. Myslím, že na půdě garančního výboru bude př…",7,"""ps""",2021,110,6,2024-07-10,"""PČR, PS 2021-...""","""110. schůze""",true,true,true,null,181
"""Poslanec Jan Lacina""","""2021ps_stenprot_112schuz_s112144.htm""","""/sqw/detail.sqw?id=6737""","""Další pozměňovací návrh, který jsem do systému nahrál pod číslem 4850, a jeho cílem je zajistit další nerušený chod Státního fondu kinematografie, nov…",1,"""ps""",2021,112,144,2024-09-13,"""PČR, PS 2021-...""","""112. schůze""",true,true,false,"""6737""",639


In [8]:
df.filter(pl.col("text").str.contains("(?i)woke")).head(5)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32


In [9]:
df.filter(pl.col("text").str.contains("(?i)narativ")).head(5)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32
"""Poslanec Jan Chvojka""","""2017ps_stenprot_019schuz_s019184.htm""","""/sqw/detail.sqw?id=5926""","""Děkuji za slovo, pane předsedo. Ještě jednou hezké dopoledne, kolegyně, kolegové, vážený pane ministře. Já se přiznám, že už jsem vystupovat nechtěl, …",3,"""ps""",2017,19,184,2018-09-19,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""19. schůze""",true,true,true,"""5926""",679
"""Poslanec Dominik Feri""","""2017ps_stenprot_039schuz_s039180.htm""","""/sqw/detail.sqw?id=6449""","""Děkuji za slovo, pane místopředsedo. Vážené kolegyně, vážení kolegové, když se podíváte na vládní stanovisko u tohoto tisku, tak uvidíte, že je velmi …",3,"""ps""",2017,39,180,2019-12-03,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""39. schůze""",true,true,true,"""6449""",224
"""Poslanec Dominik Feri""","""2017ps_stenprot_044schuz_s044128.htm""","""/sqw/detail.sqw?id=6449""","""Děkuji za slovo, pane místopředsedo. Jenom velmi krátce. Snad to mohu okomentovat, poněvadž jsem z Teplic. Vláda to nezvládla nejenom po formální strá…",10,"""ps""",2017,44,128,2020-04-09,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""44. schůze""",true,true,true,"""6449""",117
"""Ministr kultury ČR Lubomír Zaorálek""","""2017ps_stenprot_049schuz_s049230.htm""","""http://www.vlada.cz/cz/clenove-vlady/lubomir-zaoralek-115390/""","""Nemám na to moc času, ale aspoň vám naznačím, v čem je ten rozdíl, a proč si nemyslím, že to je tak úplně pravda, že jde o to, že si ti občané prostě …",1,"""ps""",2017,49,230,2020-06-03,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""49. schůze""",true,true,false,null,1723
"""Předseda vlády ČR Andrej Babiš""","""2017ps_stenprot_072schuz_s072105.htm""","""https://www.vlada.cz/cz/clenove-vlady/premier/zivotopis/andrej-babis-162063/""","""No děkuji za dotaz. Tak já jsem na ten dotaz novináře normálně upřímně odpověděl. Ano, nemám 51 let slezinu, to je hlavní imunitní orgán. Když mi bylo…",8,"""ps""",2017,72,105,2020-12-03,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""72. schůze""",true,true,true,null,625


## Skokani

In [11]:
import re

In [12]:
def ocisti(frejm):
    return frejm.with_columns(pl.col('text').str.replace_all(r'[^\w\s]','',).str.to_lowercase())

In [13]:
def zgrupni_a_pores_cetnost(frejm):

    frejm_pracovni = frejm.with_columns(pl.col('text').str.split(' ')).explode('text').group_by('text').len().sort(by='len',descending=True)
    celkem = frejm_pracovni.select(pl.col('len')).sum().item()
    
    return frejm_pracovni.with_columns((pl.col('len') / celkem).alias('podil'))

In [14]:
stara_snemovna = zgrupni_a_pores_cetnost(ocisti(df.filter(pl.col("obdobi") == 2017)))

In [15]:
stara_snemovna

text,len,podil
str,u32,f64
"""a""",396529,0.03013
"""to""",277500,0.021086
"""se""",253120,0.019233
"""je""",232138,0.017639
"""v""",220998,0.016792
"""že""",209647,0.01593
"""na""",187316,0.014233
"""o""",125001,0.009498
"""tak""",101117,0.007683


In [16]:
nova_snemovna = zgrupni_a_pores_cetnost(ocisti(df.filter(pl.col("obdobi") == 2021)))

In [17]:
porovnani = stara_snemovna.rename(
    {'podil':'podil_2017','len':'pocet_2017'}
).join(
    nova_snemovna.rename(
        {'podil':'podil_2021','len':'pocet_2021'}
    ), on='text', how='full'
).with_columns(
    (pl.col('podil_2021') / pl.col('podil_2017')).alias('rozdil')
)

In [18]:
porovnani

text,pocet_2017,podil_2017,text_right,pocet_2021,podil_2021,rozdil
str,u32,f64,str,u32,f64,f64
"""a""",396529,0.03013,"""a""",484414,0.03091,1.025874
"""to""",277500,0.021086,"""to""",373088,0.023806,1.129017
"""se""",253120,0.019233,"""se""",299082,0.019084,0.992238
"""je""",232138,0.017639,"""je""",271748,0.01734,0.983042
"""v""",220998,0.016792,"""v""",258102,0.016469,0.980743
"""že""",209647,0.01593,"""že""",244555,0.015605,0.97958
"""na""",187316,0.014233,"""na""",232179,0.014815,1.040879
"""o""",125001,0.009498,"""o""",134088,0.008556,0.9008
"""tak""",101117,0.007683,"""tak""",121183,0.007732,1.006398


In [19]:
hranice = 30
porovnani.filter((pl.col('pocet_2017') >= hranice) & (pl.col('pocet_2021') >= hranice)).sort(by='rozdil',descending=True)

text,pocet_2017,podil_2017,text_right,pocet_2021,podil_2021,rozdil
str,u32,f64,str,u32,f64,f64
"""zmáčkne""",36,0.000003,"""zmáčkne""",2121,0.000135,49.475516
"""2026""",35,0.000003,"""2026""",1125,0.000072,26.992098
"""ztišení""",39,0.000003,"""ztišení""",1051,0.000067,22.630298
"""2024""",148,0.000011,"""2024""",3644,0.000233,20.676109
"""ukrajině""",90,0.000007,"""ukrajině""",2214,0.000141,20.657952
"""zdražování""",33,0.000003,"""zdražování""",781,0.00005,19.874182
"""ukrajiny""",79,0.000006,"""ukrajiny""",1746,0.000111,18.55963
"""stiskne""",81,0.000006,"""stiskne""",1531,0.000098,15.87239
"""2025""",146,0.000011,"""2025""",2676,0.000171,15.391658


In [20]:
hranice = 100
porovnani.filter((pl.col('pocet_2017') >= hranice) & (pl.col('pocet_2021') >= hranice)).sort(by='rozdil',descending=True)

text,pocet_2017,podil_2017,text_right,pocet_2021,podil_2021,rozdil
str,u32,f64,str,u32,f64,f64
"""2024""",148,0.000011,"""2024""",3644,0.000233,20.676109
"""2025""",146,0.000011,"""2025""",2676,0.000171,15.391658
"""energií""",173,0.000013,"""energií""",3052,0.000195,14.814622
"""2023""",280,0.000021,"""2023""",4625,0.000295,13.870939
"""zvedne""",202,0.000015,"""zvedne""",2747,0.000175,11.419825
"""teďka""",103,0.000008,"""teďka""",1328,0.000085,10.827122
"""fialy""",253,0.000019,"""fialy""",2712,0.000173,9.001633
"""místopředsedkyně""",474,0.000036,"""místopředsedkyně""",5007,0.000319,8.870568
"""korespondenční""",275,0.000021,"""korespondenční""",2876,0.000184,8.782302


In [21]:
hranice = 100
porovnani.filter((pl.col('pocet_2017') >= hranice) & (pl.col('pocet_2021') >= hranice)).sort(by='rozdil',descending=False).with_columns(1 / pl.col("rozdil"))

text,pocet_2017,podil_2017,text_right,pocet_2021,podil_2021,rozdil,literal
str,u32,f64,str,u32,f64,f64,f64
"""klaus""",793,0.00006,"""klaus""",102,0.000007,0.108014,9.258078
"""faltýnek""",1047,0.00008,"""faltýnek""",155,0.00001,0.124319,8.043829
"""testy""",678,0.000052,"""testy""",111,0.000007,0.137482,7.273686
"""nesouhlasné""",2064,0.000157,"""nesouhlasné""",356,0.000023,0.144841,6.904107
"""okd""",684,0.000052,"""okd""",118,0.000008,0.14487,6.902747
"""zahájil""",3737,0.000284,"""zahájil""",670,0.000043,0.150558,6.641958
"""čssd""",1871,0.000142,"""čssd""",344,0.000022,0.154396,6.47684
"""bonusu""",688,0.000052,"""bonusu""",131,0.000008,0.159895,6.254102
"""ministryni""",2735,0.000208,"""ministryni""",542,0.000035,0.166416,6.009051


## Skokani od roku 1993

In [23]:
import simplemma

In [24]:
def prvnipad(slovo):
    if len(slovo) > 2:
        return simplemma.lemmatize(slovo, lang='cs')
    else:
        return None

In [25]:
stare_snemovny = zgrupni_a_pores_cetnost(
    ocisti(df.filter(pl.col("obdobi").is_between(1993,2020)))
).with_columns(
    pl.col("text").map_elements(prvnipad, return_dtype=pl.String).alias("lemma")
).group_by(
    "lemma"
).agg([
    pl.col("len").sum(),
    pl.col("podil").sum()
]).rename(
    {'len':'len_93_21','podil':'podil_93_21'}
)

In [26]:
snemovna_2021 = zgrupni_a_pores_cetnost(ocisti(df.filter(pl.col("obdobi") == 2021))).with_columns(
    pl.col("text").map_elements(prvnipad, return_dtype=pl.String).alias("lemma")
).group_by(
    "lemma"
).agg([
    pl.col("len").sum(),
    pl.col("podil").sum()
]).rename(
    {'len':'len_21_25','podil':'podil_21_25'}
)

In [27]:
stare_snemovny.sample(10)

lemma,len_93_21,podil_93_21
str,u32,f64
"""nekončenými""",1,1.5463e-8
"""pseudokoalice""",1,1.5463e-8
"""zažaluje""",11,1.7010e-7
"""určovaného""",2,3.0927e-8
"""mýlkám""",2,3.0927e-8
"""pominut""",13,2.0102e-7
"""dovysvětlujícím""",1,1.5463e-8
"""koncentrátu""",7,1.0824e-7
"""nevstupování""",1,1.5463e-8


In [28]:
len(stare_snemovny)

260500

In [29]:
porovnani_93_21 = stare_snemovny.join(
    snemovna_2021, how='full', on='lemma'
).with_columns(
    (pl.col('podil_21_25') / pl.col('podil_93_21')).alias('rozdil')
)

In [30]:
porovnani_93_21.sort(by='rozdil',descending=True)

lemma,len_93_21,podil_93_21,lemma_right,len_21_25,podil_21_25,rozdil
str,u32,f64,str,u32,f64,f64
"""dopravováno""",3,4.6390e-8,null,null,null,null
"""čtyřicetdevítka""",1,1.5463e-8,null,null,null,null
"""6191992""",1,1.5463e-8,null,null,null,null
"""tkáče""",1,1.5463e-8,null,null,null,null
"""lvíče""",1,1.5463e-8,null,null,null,null
"""vyznamenaného""",2,3.0927e-8,null,null,null,null
"""kluzku""",1,1.5463e-8,null,null,null,null
"""vozdeckému""",6,9.2780e-8,null,null,null,null
"""geodeta""",1,1.5463e-8,null,null,null,null


In [ ]:
df.filter(pl.col("text").str.contains("[Bb]itcoin")).sort(by='datum').head(10)

In [ ]:
porovnani_93_21.filter(pl.col('lemma_right').str.contains('menstr'))

In [ ]:
porovnani_93_21.filter(pl.col('lemma') == 'bydlení')

In [ ]:
porovnani_93_21.filter((pl.col('len_93_21') >= 5) & (pl.col('len_21_25') >= 10)).sort(by='rozdil',descending=True).head(150)

In [32]:
narust = {
    "pětikoalice":"(?i)pětikoal",
    "energie":"(?i)energ[ie]",
    "korespondenční":"(?i)korespondenčn",
    "omikron":"(?i)omikron",
    "HHC":"(?i)hhc",
    "kratom":"(?i)kratom",
    "drahota":"(?i)drahot",
    "Green Deal":"(?i)green deal",
    "Ukrajina, Ukrajinci":"(?i)ukrajin",
    "Křivoklát":"(?i)křivoklát",
    "menstruace":"(?i)menstru[oau]",
    "bitcoin":"(?i)bitco",
    "kryptoměny":"(?i)kryptom",
    "bydlení":"(?i)bydlení",
    "dozimetr":"(?i)dozimet",
    "inflace":"(?i)infla[cč]"
}

In [34]:
maxima = []
for p, n in narust.items():
    maximum = {
        'slovo':p,
        'max':
        df.filter(
            pl.col('obdobi') == 2021
        ).filter(
            pl.col('text').str.contains(n)
        ).with_columns(
            pl.col('text').str.count_matches(n).alias('pocet_vyskytu')
        ).group_by_dynamic(
            index_column='datum',every='1y'
        ).agg(
            pl.col('pocet_vyskytu').sum()
        ).select(
            pl.col('pocet_vyskytu').max()
        ).item()
    }
    maxima.append(maximum)

maxima = pl.DataFrame(maxima).sort(by='max',descending=True).select(pl.col('slovo')).to_series().to_list()

In [36]:
slov_po_roce = df.group_by_dynamic(index_column='datum',every='1y').agg(pl.col('pocet_slov').sum())

In [38]:
def cetnost_po_roce(popisek, hledani):
    return df.filter(
        pl.col('text').str.contains(hledani)
    ).with_columns(
        pl.col('text').str.count_matches(hledani).alias('pocet_vyskytu')
    ).group_by_dynamic(
        index_column='datum',every='1y'
    ).agg(
        pl.col('pocet_vyskytu').sum()
    ).join(
        slov_po_roce, on='datum', how='right'
    ).with_columns(
        (pl.col('pocet_vyskytu') / pl.col('pocet_slov')).alias('podil')
    ).select(
        pl.col(['datum','podil'])
    ).with_columns(
        pl.lit(popisek).alias('co')
    )

In [40]:
do_grafu = pl.DataFrame()
for popis, hled in narust.items():
    do_grafu = pl.concat([do_grafu,cetnost_po_roce(popis, hled)])

In [53]:
do_grafu.write_json("data_raw/ukazka_grafu_data.json")

In [49]:
graf = alt.Chart(
    do_grafu.to_pandas(),
    title=alt.Title(
        "Skokani období 2021-2025",
        subtitle=["Slova, která poslanci dropovali nejvíc","v éře samostatné České republiky."]
    ),
    height=60
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q'),
    alt.Row('co:N', sort=maxima)
)

graf

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [51]:
graf.save("data_raw/ukazka_grafu_graf.svg")

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


In [ ]:
cetnost_po_roce("pětikoalice","pětikoal")

In [ ]:
porovnani_93_21.filter(pl.col("lemma") == "lex")

In [ ]:
porovnani_93_21.filter(pl.col('podil_21_25').is_null()).sort(by='podil_93_21',descending=True).head(150)

### O čem se nemluví

In [ ]:
porovnani_93_21.filter(~pl.col('podil_93_21').is_null()).filter(pl.col('len_21_25').is_null()).sort(by='len_93_21',descending=True).head(150)

In [ ]:
porovnani_93_21.filter(~pl.col('podil_93_21').is_null()).filter(~pl.col('len_21_25').is_null()).sort(by='rozdil',descending=False).head(150)